In [8]:
# trying out 1D image compression stuff

from scipy.fftpack import dct, idct
import numpy as np

In [15]:
array1 = np.array([4., 3., 5., 10., 5., 3.])

In [19]:
dct1 = dct(array1, norm='ortho')
print(dct1)

[12.24744871 -1.00596527 -4.          3.26598632  1.73205081 -1.82246185]


In [17]:
array1recov = idct(dct1, norm='ortho')

In [18]:
print(array1recov)

[ 4.  3.  5. 10.  5.  3.]
